# Settings

In [21]:
CASE = 2
exceptions = []

match CASE:
    case 0: 
        IN_FILENAME = "../data/processed/filtered.xlsx"
        OUT_FILENAME = "../data/product/product.xlsx"
        
    case 1:
        IN_FILENAME = "../data/processed/filtered-nfv.xlsx"
        OUT_FILENAME = "../data/product/product-nfv.xlsx"
        exceptions.append("isFamiliar")
        
    case 2:
        IN_FILENAME = "../data/processed/filtered-ffu.xlsx"
        OUT_FILENAME = "../data/product/product-ffu.xlsx"

# adjustments

In [22]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (Xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [23]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

In [24]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()
columns = namedtuple("Columns", " ".join(columns.keys()))(**columns)
years = namedtuple("Years", re.sub(r'[\[\],\']', "", str([*map(lambda y: f'y{y}', years)])))(*years)

In [25]:
with open("../constants/filtered_columns.json", "r") as file:
    fcolumns = json.load(file)
    
for elm in exceptions: del fcolumns[elm]
    
fcolumns = namedtuple("FColumns", " ".join(fcolumns))(**fcolumns)

### tools

In [26]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [27]:
data = pd.read_excel(IN_FILENAME)

In [28]:
y_years = [2022]

df = pd.DataFrame(columns=[*fcolumns._asdict().keys()])

for row in data.iloc:
	df = pd.concat([
		df, pd.DataFrame([[row[YYYY(col, year if col != fcolumns.z_cat else year+2) ]  for col in fcolumns] for year in y_years], columns=[*fcolumns._asdict().keys()])
		], ignore_index=True)


/var/folders/cs/8k4rd8wn11g9t0b617_34rdr0000gp/T/ipykernel_6803/2147180622.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([


In [29]:
df.head()

,TD_div_TA,WC_div_TA,EBITDA_div_TA,lnTA,isFamiliar,z_cat
0,0.625639,0.191468,0.030919,19.710178,0.0,1.0
1,0.140025,0.292908,0.178364,18.660925,0.0,0.0
2,0.221861,0.370248,0.090879,18.385202,1.0,0.0
3,0.635556,0.292989,0.064907,17.903484,1.0,1.0
4,0.294878,0.317507,0.026602,17.891653,1.0,0.0


# Save dataframe

In [30]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")